In [ ]:
# This code reads AD matrix and DP matrix for inputs of VAE and visualization of BPDCN dataset

import torch
import numpy as np

from scipy.io import mmread
AD = mmread('cellSNP.tag.AD.mtx')
DP = mmread('cellSNP.tag.DP.mtx')

threshold_var = 1
SNP = AD / DP

mean_matrix = np.outer(np.nanmean(SNP, 1), np.ones(SNP.shape[1]))
nan = np.isnan(SNP)
SNP[nan] = mean_matrix[nan]
SNP = torch.tensor(SNP.T).float()

allgene_var = torch.var(torch.logit(SNP, eps = 0.01), 0).numpy()
SNP = SNP[:, allgene_var > threshold_var]
depth = torch.tensor(DP.todense()[allgene_var > threshold_var, :].T)

In [ ]:
# This code reads VCF file of BPDCN dataset

import io
import os
import pandas as pd

def read_vcf(path):
    
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
        
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

information = read_vcf("cellSNP.base.vcf")